In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
import ast
import warnings
warnings.filterwarnings('ignore')
from natsort import natsorted
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))
import time
today = time.strftime("%Y_%m_%d")

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

# importing the sys module
import sys        
 
# appending the directory of mod.py
# in the sys.path list
sys.path.append('C:/Users/groupby/Documents/GitHub/SQL Extract And Apply') 
from enrich_dimensions.rounds import rounds, rounding,re_extract, curate, round_string_float,reg_ex,clean_data#,rounding_kimball
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

# Find unusual units
def rounding_kimball(heights,col, texto):
    regex_pattern_rounding=rf'''(?i)((?<!\d)(?<!\.)0(?!\.))|(\.0(?!\d))|(\.\d0(?<!\d))|(xa)|(\\)|([{texto}])|(\')|((?<![A-z])"\])|(\d(?<![A-z])")|("(?<!\d).{0,3}[A-z]")|((?<=[A-z])\s")|((?<=[A-z])\.)|((?<!\d)000)|((?<!\d\s)in)|((?<!\w)(?<!\.)(?<!")0(?!\.))|((?<!")\,)|(\,(?!"))|((?<!")\])|((?<!\w)(?<!\.)(?<!")\d(?!.\w)(?!\d)(?!\.))|(\-|ï|¿|½)|(ft.{0,4}in)|(\(|\))|("\]"\])|(\["\[")|(in\d+)|(?<=\[)"(?=\])|((?i)inft)|((?i)ftin)|((?<=")(?:ft|in))|(\d+\..{3,4}\/)|()'''         
    heights['rounding'] = heights[col].apply(lambda x: re_extract(regex_pattern_rounding, x))
    print('Number of SKUs that need Rounding: '+ str(len(heights[heights['rounding'].astype(str)!='[]'])))
    return heights[heights['rounding'].astype(str)!='[]'] 

# Curated BJs

In [2]:
customer_id = '66'
customer_name='%bjs%'

dateszs='2001-07-01'
dateszsz='2023-05-10'
params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs,
         'dateszsz':dateszsz}
bj = query_from_file(file_name='../query/curated_all_attributes_date.sql', params=params)
# dfs=dfs[dfs['resolution'].astype(str)!='rules']
print(len(bj))

384908


In [3]:
del bj['customer_name']
# del bj['time']
# del bj['family_friendly']
# del motion['curated_date']
del bj['curation_tasks.curated_by']

In [11]:
fill_dict = bj.groupby('external_id')['buckets'].last().to_dict()
bj['buckets'] = bj['buckets'].fillna(bj['external_id'].map(fill_dict))

In [45]:
diameter=bj[bj['attribute'].astype(str)=='diameter']

In [44]:
bj['attribute'].explode().value_counts()

product_type                   56151
search_product_type            48339
color                          35736
features                       24497
fabric                         19598
quantity                       17226
pattern                        16352
sleeve_length                  10781
concern                         9573
material                        9323
neckline                        8452
primary_flavor                  6801
closure                         5923
gender                          5437
metal_type                      4861
indoor_outdoor                  4651
gold_karat                      4527
stone_type                      4518
pearl_type                      4487
carat_total_weight              4486
stone_color                     4475
stone_shape                     4466
height                          3994
finish                          3892
supplement_type                 3874
length                          3732
pant_style                      3569
r

In [12]:
height=bj[bj['attribute'].astype(str)=='height']
width=bj[bj['attribute'].astype(str)=='width']
depth=bj[bj['attribute'].astype(str)=='depth']
weight=bj[bj['attribute'].astype(str)=='weight']
length=bj[bj['attribute'].astype(str)=='length']

In [13]:
print(len(height))
height['value'].explode().value_counts()

3994


n/a          251
24 in.       176
8 in.        145
12 in.       143
14 in.       119
            ... 
44.09 in.      1
39.88 in.      1
48.75 in.      1
28.45 in.      1
67.25 in.      1
Name: value, Length: 600, dtype: int64

In [14]:
print(len(width))
width['value'].explode().value_counts()

2946


n/a          145
20 in.       104
48 in.       103
24 in.        83
18 in.        66
            ... 
5.31 in.       1
29.8 in.       1
7.6 in.        1
44.25 in.      1
6.3 in         1
Name: value, Length: 581, dtype: int64

In [30]:
print(len(depth))
depth['value'].explode().value_counts()[0:10]

403


n/a         80
24.5 in.    19
28 in.      15
31 in.      14
16 in.      14
18 in.      10
27.9 in.     9
29.9 in.     8
33.7 in.     6
27 in.       6
Name: value, dtype: int64

In [16]:
print(len(weight))
weight['value'].explode().value_counts()

9


30 lbs     1
41 lbs     1
32 lbs     1
110 lbs    1
280 lbs    1
40 lbs     1
6 lb       1
200 lb     1
100 lbs    1
Name: value, dtype: int64

# Height

In [31]:
pat='''(?i)(Dimensions?.{0,7}\d+.{0,3}(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?.{0,3}(?:''|"|”|in\.?){0,3}.{0,3}x\s?\d+.{0,3}(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?.{0,3}(?:''|"|”|in\.?){0,3}.{0,3}x\s?\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.){0,3}\'?)|(7'.{0,30}christmas tree)|(\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.?){0,3}.?(?:L?(?:ength)?|d(?:epth)?)?.?x.?\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.?){0,3}.?(?:W(?:idth)?)?.?X.?\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.?){0,3}.?(?:H(?:eight|igh)?)?)|(Height.{0,19}'\:\s?'\d+(?:\.\d+)?")|(\d+'\s?x\s?\d+'(?!\sx))|()'''         
height['m_long']=height['long_desc'].apply(lambda x: re_extract(pat, str(x)))
height['m_name']=height['product_name'].apply(lambda x: re_extract(pat, str(x)))
height['m_custom']=height['custom_fields'].apply(lambda x: re_extract(pat, str(x)))

In [32]:
h=height[((height['m_long'].astype(str)!='[]')|(height['m_long'].astype(str)!='[]')|(height['m_long'].astype(str)!='[]'))&(height['value'].astype(str)=='n/a')]
he=height[((height['m_long'].astype(str)!='[]')|(height['m_long'].astype(str)!='[]')|(height['m_long'].astype(str)!='[]'))]
print(len(height))
print(len(height[height['value'].astype(str)=='n/a']))
print(len(h))
print(len(he))

3994
251
11
356


In [33]:
hei=height[((height['m_long'].astype(str)=='[]')&(height['m_long'].astype(str)=='[]')&(height['m_long'].astype(str)=='[]'))&(height['value'].astype(str)!='n/a')]
print(len(hei))

3398


In [34]:
p='''(?i)(.{0,20}(?:(?:item|peak).?height(?:.{0,3}\'.{0,3}\:.{0,3}\'.{0,3})?)\d+(?:\.\d+)?\s?(?:''|"|”|in\.?).{0,20})|(.{0,20}\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:\s?(?:''|"|”|in\.)\s?H(?:igh|eight|\\n)?)(?! draw)(?!i density)(?!.tens)(?![A-z]).{0,20})|(height.{0,5}\d+(?:\.\d+)?\s?(?:''|"|”|in\.))|(\d+'\s?Aluminum Umbrella)|(?i)(\d+.?(?:\d*(?:\/\d+)?).?'\s?\:\s?\'?\s?Item.?height)|(Dimensions?.{0,7}\d+.{0,3}(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?.{0,3}(?:''|"|”|in\.?){0,3}.{0,3}x\s?\d+.{0,3}(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?.{0,3}(?:''|"|”|in\.?){0,3}.{0,3}x\s?\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.){0,3}\'?)|(7'.{0,30}christmas tree)|(\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.?){0,3}.?(?:L?(?:ength)?|d(?:epth)?)?.?x.?\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.?){0,3}.?(?:W(?:idth)?)?.?X.?\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.?){0,3}.?(?:H(?:eight|igh)?)?)|(Height.{0,19}'\:\s?'\d+(?:\.\d+)?")|(\d+'\s?x\s?\d+'(?!\sx))|()'''
hei['custom_fields'].apply(lambda x: re_extract(p,str(x)))

26                                                                                  [dimensions': '83.5"L x 72"W x 13.5"]
74                                                                                      [dimensions': '42"L x 74"W x 10"]
103                                                                                                                    []
302                                                                                  [dimensions': '4.25"L x 4.25"W x 9"]
563                                                                                  [dimensions': '76"L x 11.8"W x 5.9"]
                                                               ...                                                       
382290                                                                                      [14' x 10', 14?L x 10?W x 8?]
382304                                                        [nual Retractable', 'Item_height': '7"', 'Item_weight': '6]
382314     [ '5.11"L x 5

In [36]:
# height['match']=height['custom_fields'].apply(lambda x: re_extract(p,str(x)))

In [ ]:
# x=height[height['match'].astype(str)!='[]']
# y=height[(height['match'].astype(str)=='[]')&(height['value'].astype(str)!='n/a')&(height['m_name'].astype(str)=='[]')&(height['m_long'].astype(str)=='[]')]
# print(len(x))
# print(len(y))

In [ ]:
# y=height[((height['match'].astype(str)!='[]')|(height['m_name'].astype(str)!='[]')|(height['m_long'].astype(str)!='[]'))&(height['value'].astype(str)=='n/a')]

In [ ]:
# print(len(y))

In [ ]:
# na=height[height['value'].astype(str)=='n/a']
# print(len(na))

In [ ]:
# y[['external_id','value','m_long','m_custom','match']]

In [37]:
pat='''(?i)([A-z])|()'''
height['units']=height['value'].apply(lambda x: re_extract(pat,str(x)))
units=height[height['units'].astype(str)=='[]']

In [38]:
units['value'].explode().value_counts()

Series([], Name: value, dtype: int64)

# Width

In [39]:
pat='''(?i)([A-z])|()'''
width['units']=width['value'].apply(lambda x: re_extract(pat,str(x)))
unit=width[width['units'].astype(str)=='[]']
unit['value'].explode().value_counts()

Series([], Name: value, dtype: int64)

In [199]:
p='''(?i)(.{0,20}(?:(?:item|peak).?(?:width|wide)(?:.{0,3}\'.{0,3}\:.{0,3}\'.{0,3})?)\d+(?:\.\d+)?\s?(?:''|"|”|in\.?).{0,20})|(.{0,20}\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:\s?(?:''|"|”|in\.)\s?W(?:idth|ide|\\n)?)(?! draw)(?!i density)(?!.tens)(?![A-z]).{0,20})|((?:width|wide).{0,5}\d+(?:\.\d+)?\s?(?:''|"|”|in\.))|(\d+'\s?Aluminum Umbrella)|(?i)(\d+.?(?:\d*(?:\/\d+)?).?'\s?\:\s?\'?\s?Item.?(?:width|wide))|(Dimensions?.{0,7}\d+.{0,3}(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?.{0,3}(?:''|"|”|in\.?){0,3}.{0,3}x\s?\d+.{0,3}(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?.{0,3}(?:''|"|”|in\.?){0,3}.{0,3}x\s?\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.){0,3}\'?)|(7'.{0,30}christmas tree)|(\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.?){0,3}.?(?:L?(?:ength)?|d(?:epth)?)?.?x.?\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.?){0,3}.?(?:W(?:idth|ide)?)?.?X.?\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.?){0,3}.?(?:H(?:eight|igh)?)?)|(Height.{0,19}'\:\s?'\d+(?:\.\d+)?")|(\d+'\s?x\s?\d+'(?!\sx))|()'''         
width['match']=width['custom_fields'].apply(lambda x: re_extract(p,str(x)))

In [226]:
width['m_name']=width['product_name'].apply(lambda x: re_extract(p,str(x)))
width['m_long']=width['long_desc'].apply(lambda x: re_extract(p,str(x)))

In [227]:
print(len(width))
x=width[width['match'].astype(str)!='[]']
print(len(x))

2944
2714


In [233]:
y=width[(width['match'].astype(str)=='[]')&(width['m_long'].astype(str)=='[]')&(width['m_name'].astype(str)=='[]')&(width['value'].astype(str)!='n/a')]
print(len(y))

97


In [284]:
x=width[((width['match'].astype(str)!='[]')&(width['m_long'].astype(str)!='[]')&(width['m_name'].astype(str)!='[]'))&(width['value'].astype(str)=='n/a')]
print(len(x))

5


In [287]:
x['Q:width']='[]'
match_width_wipe=x[['external_id','Q:width']]

In [239]:
y['value'][0:15]

2989        30 in.
6249        16 in.
12140       30 in.
16179       94 in.
22036    27.75 in.
53670      9.5 in.
66424    15.25 in.
96430       18 in.
96596       15 in.
96680       18 in.
96781       20 in.
96984       18 in.
97012       16 in.
97117       18 in.
97155       16 in.
Name: value, dtype: object

In [243]:
print(len(y))

97


In [255]:
y=y.reset_index()

In [394]:
# for i in range(len(y)):
#     print(y['value'][i])
#     print(y['custom_fields'][i])
#     print('')
#     print('')

In [289]:
# y[['external_id','value','m_long','m_name','match','custom_fields']]['custom_fields']

In [290]:
# x['match'].explode().value_counts()[0:500]
# x[['external_id','value','match',str('custom_fields')]]

# Depth

In [40]:
pat='''(?i)([A-z])|()'''
depth['units']=depth['value'].apply(lambda x: re_extract(pat,str(x)))
uni=depth[depth['units'].astype(str)=='[]']
uni['value'].explode().value_counts()

Series([], Name: value, dtype: int64)

In [318]:
p='''(?i)(.{0,20}(?:(?:item|peak).?(?:depth)(?:.{0,3}\'.{0,3}\:.{0,3}\'.{0,3})?)\d+(?:\.\d+)?\s?(?:''|"|”|in\.?).{0,20})|(.{0,20}\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:\s?(?:''|"|”|in\.)\s?D(?:epth|\\n)?)(?! draw)(?!i density)(?!.tens)(?![A-z]).{0,20})|((?:depth).{0,5}\d+(?:\.\d+)?\s?(?:''|"|”|in\.))|(\d+'\s?Aluminum Umbrella)|(?i)(\d+.?(?:\d*(?:\/\d+)?).?'\s?\:\s?\'?\s?Item.?(?:depth))|(.{0,10}\d+.{0,3}(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?.{0,3}\s?[A-z]{0,10}(?:''|"|”|in\.?){0,3}.{0,3}x\s?\d+.{0,3}(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?.{0,3}[A-z]{0,10}(?:''|"|”|in\.?){0,3}.{0,3}x\s?\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.){0,3}.{0,3}[A-z]{0,10}\'?)|(7'.{0,30}christmas tree)|(\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.?){0,3}.?(?:L?(?:ength)?|d(?:epth)?)?.?x.?\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.?){0,3}.?(?:W(?:idth|ide)?)?.?X.?\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.?){0,3}.?(?:H(?:eight|igh)?)?)|(Depth(?:.with_door_open_90|.closed_excluding_)?.{0,15}handles.{0,10}'?\:?\s?'\d+(?:\.\d+)?")|(\d+'\s?x\s?\d+'(?!\sx))|(Depth(?:.with_door_open_90|.closed_excluding_|_closed_including)?.{0,15}'?\:?\s?'\d+(?:\.\d+)?)|()'''         
depth['match']=depth['custom_fields'].apply(lambda x: re_extract(p,str(x)))

In [319]:
depth['m_long']=depth['long_desc'].apply(lambda x: re_extract(p,str(x)))
depth['m_name']=depth['product_name'].apply(lambda x: re_extract(p,str(x)))

In [320]:
print(len(depth))
print(len(depth[depth['value'].astype(str)=='n/a']))
x=depth[depth['match'].astype(str)!='[]']
print(len(x))

402
80
362


In [322]:
depth[depth['value'].astype(str)=='n/a'][['value','external_id','match','custom_fields']]['external_id']

22057     3000000000002374326
48784     3000000000003306760
49025     3000000000003067290
49083     3000000000004317766
49180     3000000000003305756
49385     3000000000003306800
49501     3000000000004317762
49653     3000000000003306326
49963     3000000000003306878
49971     3000000000000409293
50150     3000000000000409291
50158     3000000000003306830
50257                  130417
50408     3000000000003306276
50483     3000000000003434750
50500     3000000000003306872
50566     3000000000003089292
50575     3000000000003306788
50754     3000000000000099110
50803     3000000000004215786
50893     3000000000003306328
50930     3000000000003306890
50936     3000000000003306258
51045     3000000000002279254
51113                  130352
51275     3000000000004317760
51278     3000000000004215820
51385     3000000000002271288
51437     3000000000003306260
51522     3000000000004215776
51548     3000000000003306860
51620     3000000000004215798
51638                  131044
51828     

In [390]:
x=depth[(depth['value'].astype(str)=='n/a')&(depth['match'].astype(str)!='[]')]
print(len(x))

60


In [391]:
x['Q:depth']='[]'
match_depth_wipe=x[['external_id','Q:depth']]

# weight

In [22]:
pat='''(?i)(.{0,30}(?:\d+.{0,7}(?:lbs?|pound|weight)).{0,30})|()'''
weight['match']=weight['custom_fields'].apply(lambda x: re_extract(pat,str(x)))

In [23]:
weight['m_long']=weight['long_desc'].apply(lambda x: re_extract(pat,str(x)))
weight['m_name']=weight['product_name'].apply(lambda x: re_extract(pat,str(x)))

In [24]:
print(len(weight))

9


In [25]:
na=weight[weight['value'].astype(str)=='n/a']
print(len(na))

0


In [26]:
match=weight[(weight['match'].astype(str)!='[]')|(weight['m_long'].astype(str)!='[]')|(weight['m_name'].astype(str)!='[]')]
print(len(match))

9


In [27]:
matches=weight[((weight['match'].astype(str)!='[]')|(weight['m_long'].astype(str)!='[]')|(weight['m_name'].astype(str)!='[]'))&(weight['value'].astype(str)=='n/a')]
print(len(matches))

0


# length

In [41]:
pat='''(?i)([A-z])|()'''
length['units']=length['value'].apply(lambda x: re_extract(pat,str(x)))
un=length[length['units'].astype(str)=='[]']
un['value'].explode().value_counts()

Series([], Name: value, dtype: int64)

In [43]:
length['value'].explode().value_counts()

n/a                    998
48 in.                  89
72 in.                  82
58 in.                  67
24 in.                  64
                      ... 
["21 in.","79 in."]      1
7.75 in                  1
1.28 in.                 1
76 in.                   1
49.75 in.                1
Name: value, Length: 663, dtype: int64

In [492]:
p='''(?i)(.{0,20}(?:(?:item|peak).?(?:length)(?:.{0,3}\'.{0,3}\:.{0,3}\'.{0,3})?)\d+(?:\.\d+)?\s?(?:''|"|”|in\.?).{0,20})|(.{0,20}\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:\s?(?:''|"|”|in\.)\s?D(?:ength|ong|\\n)?)(?! draw)(?!i density)(?!.tens)(?![A-z]).{0,20})|((?:length).{0,5}\d+(?:\.\d+)?\s?(?:''|"|”|in\.))|(\d+'\s?Aluminum Umbrella)|(?i)(\d+.?(?:\d*(?:\/\d+)?).?'\s?\:\s?\'?\s?Item.?(?:length))|(.{0,10}\d+.{0,3}(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?.{0,3}\s?[A-z]{0,10}(?:''|"|”|in\.?){0,3}.{0,3}x\s?\d+.{0,3}(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?.{0,3}[A-z]{0,10}(?:''|"|”|in\.?){0,3}.{0,3}x\s?\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.){0,3}.{0,3}[A-z]{0,10}\'?)|(7'.{0,30}christmas tree)|(\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.?){0,3}.?(?:L?(?:ength)?|d(?:ength|ong)?)?.?x.?\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.?){0,3}.?(?:W(?:idth|ide)?)?.?X.?\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.?){0,3}.?(?:H(?:eight|igh)?)?)|(length(?:.with_door_open_90|.closed_excluding_)?.{0,15}handles.{0,10}'?\:?\s?'\d+(?:\.\d+)?")|(\d+'\s?x\s?\d+'(?!\sx))|(length(?:.with_door_open_90|.closed_excluding_|_closed_including)?.{0,15}'?\:?\s?'\d+(?:\.\d+)?)|()'''         
pats='''(?i)((?:(?:item|peak).?(?:(?<!cord.)(?<!cord)length)(?:.{0,3}\'.{0,3}\:.{0,3}\'.{0,3})?)\d+(?:\.\d+)?\s?(?:''|"|”|in\.?))|(\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:\s?(?:''|"|”|in\.)\s?L(?:ength|ong|\\n)?)(?! draw)(?!i density)(?!.tens)(?![A-z]))|((?:(?<!cord.)(?<!cord)length).{0,5}\d+(?:\.\d+)?\s?(?:''|"|”|in\.))|(\d+'\s?Aluminum Umbrella)|(?i)(\d+.?(?:\d*(?:\/\d+)?).?'\s?\:\s?\'?\s?Item.?(?:(?<!cord.)(?<!cord)length))|(.{0,10}\d+.{0,3}(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?.{0,3}\s?L[A-z]{0,10}(?:''|"|”|in\.?){0,3}.{0,3}x\s?\d+.{0,3}(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?.{0,3}[A-z]{0,10}(?:''|"|”|in\.?){0,3}.{0,3}x\s?\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.){0,3}.{0,3}[A-z]{0,10}\'?)|(7'.{0,30}christmas tree)|(\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.?){0,3}[^A-z]?(?:L?(?:ength)?)?.?x.?\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.?){0,3}.?(?:W(?:idth|ide)?)?.?X.?\d+(?:(?:\.|\-|\s)?\d+(?:\/\d+)?)?\s?(?:''|"|”|in\.?){0,3}.?(?:H(?:eight|igh)?)?)|((?<!cord.)(?<!cord)length(?:.with_door_open_90|.closed_excluding_)?.{0,15}handles.{0,10}'?\:?\s?'\d+(?:\.\d+)?")|(\d+'\s?x\s?\d+'(?!\sx))|((?<!cord.)(?<!cord)length(?:.with_door_open_90|.closed_excluding_|_closed_including)?.{0,15}'?\:?\s?'\d+(?:\.\d+)?)|()'''
length['match']=length['custom_fields'].apply(lambda x: re_extract(p,str(x)))
length['matches']=length['custom_fields'].apply(lambda x: re_extract(pats,str(x)))

In [493]:
length['m_long']=length['long_desc'].apply(lambda x: re_extract(p,str(x)))
length['m_name']=length['product_name'].apply(lambda x: re_extract(p,str(x)))

In [494]:
print(len(length))
print(len(length[length['value'].astype(str)=='n/a']))

3730
996


In [495]:
print(len(length[(length['value'].astype(str)=='n/a')&(length['match'].astype(str)!='[]')]))

693


In [496]:
lst=['3000000000003021785','3000000000002523302','3000000000002695269', '3000000000002695271', '3000000000002695266', '3000000000002695267', '3000000000002695265', '3000000000002695270','3000000000002364788','3000000000002090766', '3000000000002090767', '3000000000002090768', '3000000000001830748','3000000000001784272', '3000000000001784262','31629','3000000000001784272', '3000000000001784262','3000000000003322285', '3000000000003322284','3000000000002090766', '3000000000002090767', '3000000000002090768', '3000000000001830748','280157','3000000000002090766', '3000000000002090767', '3000000000002090768', '3000000000001830748','3000000000003322285', '3000000000003322284','187160','3000000000002090766', '3000000000002090767', '3000000000002090768', '3000000000001830748','3000000000003106315', '3000000000003106314','3000000000003106315', '3000000000003106314','187116','280163','280161','280165','280173','280171','280169']                                                                           
y=length[length['external_id'].isin(lst)]

In [497]:
print(len(y))

29


In [498]:
y=length[(length['value'].astype(str)=='n/a')&(length['match'].astype(str)!='[]')][['value','external_id','match','matches','custom_fields']]#[0:500]

In [499]:
# print(len(y[y['external_id'].isin(lst)]))
# y[y['external_id'].isin(lst)]
print(len(y))
# y['matches'].explode().value_counts()[0:500]

693


In [500]:
# y[['external_id','match','matches']][0:500]

In [501]:
z=y[y['matches'].astype(str)!='[]']
print(len(z))

464


In [502]:
# z['Q:length']=z['matches'].apply(lambda x: re.sub(r'(.+\:)|(L.+)|(\\)','',str(x)))

In [503]:
# print(z['matches'])

In [504]:
pattern='''(length'?\:?\s?\'?)|()'''
z['pattern']=z['matches'].apply(lambda x: re_extract(pattern,str(x)))

In [505]:
a=z[z['pattern'].astype(str)!='[]']
b=z[z['pattern'].astype(str)=='[]']

In [506]:
a=a.reset_index()
b=b.reset_index()

In [507]:
a['Q:length']=a['matches'].apply(lambda x: re.sub(r'''(?i)((?<=\[)[^l]+'?l(?:ength)\\?'?\:?\s?\'?\\?\'?)|("[^len]+(?=\]))|(L\s?x[^l]+(?=\]))|(\s?\'\s?length\\?\'?\:\s?\\?\')|(\'nsions\\?\'\:\s?\\?\')''','',str(x))).apply(lambda x: re.sub(r'(?i)(L[^lengt]+(?=\,))|(item.?length\\?\s?\'?\:?\s?\\?\'?)','',str(x))).apply(lambda x: re.sub(r'''(?i)(\'\,\s?\'pacity.+(?=\]))''','',str(x))).apply(lambda x: re.sub(r'''(?i)((?:"|\')\s?\,\s?(?:"|')?)''','","',str(x))).apply(lambda x: re.sub(r'''(\[(?:"|')?)''','["',str(x))).apply(lambda x: re.sub(r'''((?:"|')?\])''','"]',str(x))).apply(lambda x: re.sub(r'""','"',str(x))).apply(lambda x: re.sub(r'\.875','.88',str(x))).apply(lambda x: re.sub(r'','',str(x))).apply(lambda x: re.sub(r'','',str(x))).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: natsorted(x)).apply(lambda x: re.sub(r'''(((?:"|')?\s?\,\s?(?!")(?:')?))''','","',str(x))).apply(lambda x: re.sub(r'''((?<=\[)\s?'\s?)|(\s?'\s?(?=\]))|(\s?'\s?(?=\,))|((?<=\,)\s?'\s?)''','"',str(x))).apply(lambda x: re.sub(r'''("\])''',' in."]',str(x)))                                             


In [510]:
print(len(b))

421


In [554]:
b['Q:length']=b['matches'].apply(lambda x: re.sub(r'(?i)("?.?x.?\d+(?:(?:\.|\-)?\d*\/\d+|\.\d+)?"?\s?[A-z]?\\?.?x.?\d+(?:(?:\.|\-)?\d*\/\d+|\.\d+)?"?\s?[A-z]?\\?\'{0,3})|((?:nsions|length|ize_s_|Stacked)\\?\'?\s?\:\s?\\?\'?)|(\s?(?:measures\s?|\(packaged))','',str(x))).apply(lambda x: re.sub(r'''(\[(?:"|')?)''','["',str(x))).apply(lambda x: re.sub(r'''((?:"|')?\])''','"]',str(x))).apply(lambda x: re.sub(r'"\s?L\,"?','"',str(x))).apply(lambda x: re.sub(r'"?\s?L\s?"?','"',str(x))).apply(lambda x: re.sub(r'\.875','.88',str(x))).apply(lambda x: re.sub(r'''("?\s?\,\s?(?:"|'))''','","',str(x)))                                                          
s='''(?i)(\["\d+(?:\.\d+)?"\])'''
b['m']=b['Q:length'].apply(lambda x: re_extract(s,str(x)))

In [555]:
c=b[b['m'].astype(str)!='[]']
d=b[b['m'].astype(str)=='[]']

In [556]:
c['Q:length']=c['Q:length'].apply(lambda x: re.sub(r'"\]',' in."]',str(x))).apply(lambda x: re.sub(r'\.254','.25',str(x))).apply(lambda x: re.sub(r'\.254 ','.25',str(x))).apply(lambda x: re.sub(r'\.125','.13',str(x))).apply(lambda x: re.sub(r'\.095','.1',str(x)))                                

In [564]:
print(len(c))
print(len(d))

191
230


In [565]:
d['Q:length']='[]'
match_length_wipes=d[['external_id','Q:length']]

In [559]:
match_legth_two=c[['external_id','Q:length']]

In [562]:
# c[['external_id','Q:length','matches']] 

In [536]:
for i in range(len(b['Q:length'])):
    print(b['Q:length'][i])

["17"]
["3.88"]
["12","]
["63"]
["53"]
["53"]
["53"]
["63"]
["75"]
["63"]
["17.4"]
["63"]
["4.86"]
["54"]
["53"]
["68"]
["24"]
["20"]
["53"]
["64"]
["68"]
["20"]
["75"]
["64"]
["63"]
["24"]
["15.25"]
["57"]
["10' X 9'","8.27"]
["9' x 7'"]
["90"]
["84"]
["2.58"]
["26"]
["23"]
["29.8"]
["40.6"]
["23.6"]
["23.4"]
["27"]
["50"]
["15.75","12"]
["14"]
["25.25"]
["19"]
["18"]
["15.5"]
["16"]
["19"]
["15"]
["13.5","15.75"]
["15.75","12"]
["15"]
["19"]
["18"]
["13.5","15.75"]
["25.5"]
["13"]
["15.75","12.5"]
["15"]
["25.25"]
["13.5","15.75"]
["13"]
["18","17.5"]
["15.75","12"]
["18","17.5"]
["18"]
["15.5"]
["16"]
["25"]
["4'x8'","4' x 8'"]
["4'x8'","4' x 8'"]
["18"]
["18"]
["20.08"]
["4'x8'","4' x 8'"]
["18"]
["-H"]
["36"]
["18","18","18-5-"]
["18"]
["18"]
["18"]
["23"]
["18","18-5-'","18"]
["18"]
["36nch"]
["24-4-'","24","24"]
["18"]
["25-'","25"]
["54","54-"]
["54","54-"]
["23-"]
["59-'","59"]
["54","54-"]
["54","54-"]
["25-'","25"]
["19","19-"]
["25-'","25"]
["25-'","25"]
["59-'","59"]
["19"

In [509]:
match_len_un=a[['external_id','Q:length']]

In [198]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BJ --{get_df_name(matches)}match_height_wipe.csv',index=False) 
looks_good('BJ', match_height_wipe) 

In [288]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BJ --{get_df_name(matches)}match_width_wipe.csv',index=False) 
looks_good('BJ', match_width_wipe) 

In [393]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BJ --{get_df_name(matches)}match_depth_wipe.csv',index=False) 
looks_good('BJ', match_depth_wipe) 

In [563]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BJ --{get_df_name(matches)}match_legth_two.csv',index=False) 
looks_good('BJ', match_legth_two) 

In [567]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BJ --{get_df_name(matches)}match_length_wipes.csv',index=False) 
looks_good('BJ', match_length_wipes) 